In [ ]:
%run default-imports.ipynb

In [ ]:
%run run-experiment.ipynb

In [ ]:
# get explanation features of best performing model
# train the model on SINAI using all of the top features from the model
# remove the features one by one (recursive feature elimination) (iRFE)
# apply recursive feature elimination to current sinai data ()

In [ ]:
''' define options to run experiments on '''
options = {
    'target' : ["AKI"],
    'cohort' : ['MIMIC'],
    'test_size' : [0.2],
    'imputation_method' : [imputers.DEFAULT],
    'algorithm' : [algorithms.RF],
    'sampling_method': [None],
    'scale_method': [None],
    'optimize_mode': [False],
    'num_features': [50], 
    'crossval': [True],
    'explainers' : [['MimicExplainer', 'ShapExplainer', 'FeatContribExplainer't, 'LimeExplainer']]
    #'explainers' : [['MimicExplainer', 'ShapExplainer', 'FeatContribExplainer']]
    #'explainers' : [['ShapExplainer', 'FeatContribExplainer']]
    #'explainers' : [['FeatContribExplainer']]
}


In [ ]:
#experiments = unpickle('experiments.json') or {}
#exp_id = list(sorted([(k,v['performance']['discrimination']['auc']) for k,v in experiments.items()], key=lambda x: x[1], reverse=True))[:1][0][0]
#weighted_explanations = experiments[exp_id]['weighted_explanations']
#%run feature-list.ipynb
#top_features

In [ ]:
rfe_experiments = unpickle('rfe_experiments.json') or {}

In [ ]:
max_features = 10
number_features = 10000
previous_experiment = {}
n_experiments = 0

while number_features > max_features:
    ''' iterate over different options '''
    try:
        params = dict(zip(options.keys(), list(next(product(*options.values())))))        
        if previous_experiment:
            features_to_select = [exp[0] for exp in previous_experiment['weighted_explanations']]
            del features_to_select[0]
            print(f'Selected features: {len(features_to_select)}')
            params['features_to_select'] = features_to_select
            number_features = len(features_to_select)
            params['num_features'] = number_features
        experiment = run_experiment(params)
        exp_id = str(uuid.uuid1())
        rfe_experiments[exp_id] = experiment
        previous_experiment = experiment            
        n_experiments += 1
    except Exception as e:
        print(f"Could not run experiment with params {str(params)}. Error: {str(e)}")

In [ ]:
''' store everything '''
if pickle(rfe_experiments, 'rfe_experiments.json'):
    print('Successfully saved.')